# Preparing the Voxforge database

This notebook will demonstrate how to prepare the free [Voxforge](http://www.voxforge.org/) database for training. This database is a medium sized (~80 hours) database available online for free under the GPL license. A much more common database used in most research is the [TIMIT](https://catalog.ldc.upenn.edu/LDC93S1), but that costs $250 and also much smaller (~4h - although much more professionally developed than Voxforge). The best alternative today is the [Librispeech](http://www.openslr.org/12/) database, but that has a few dozen GB of data (almost 1000h) and wouldn't be sensible for a simple demo. So Voxforge it is...

First thing to do is realize what a speech corpus actually is: in its simplest form it is a collection of audio files (containing preferably speech only) with a set of transcripts of the speech. There are a few extensions to this that are worth noting:
  * phonemes - transcripts are usually presented as a list of words - although not a rule, it is often easier to start the recognition process with phonemes and go from there. Voxforge defines a list of 39 phonemes (+ silence) and contains a lexicon mapping the words into phonemes (more about that below)
  * aligned speech - the transcripts are usually just a sequence of words/phonemes, but they don't denote which word/phoneme occurs when - there are models that can learn from that (seq. learning, e.g. CTC, attention models), but having alignments is usually a big plus. TIMIT was hand-aligned by a group of professionals (which is why its a popular resource for research), but Voxforge wasn't. Fortunately, we can use one of the many available tools to do this automatically (with a margin of error - more on that below)
  * meta-data - each recording session in the Voxforge database contains a readme file with useful information about the speaker and the environment that the recording took place in. When making a serious speech recognizer, this information can be very useful (e.g. for speaker adaptation - taking into account the speaker id, gender, age, etc...)
  
## Downloading the corpus

To start working with the corpus, it needs to be downloaded first. All the files can be found in the download section of the Voxforge website under this URL:

http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit/

There are 2 versions of the main corpus: sampled at 16kHz and 8kHz. The 16 kHz one is of better quality and is known as "desktop quality speech". While the original recordings were made at an even higher quality (44.1 kHz), 16k is completely sufficient for recoginzing speech (higher quality doesn't help much). 8 kHz is known as the telephony quality and is a standard value for the old (uncompressed, aka T0) digital telephone signal. If you are making a recognizer that has to work in the telephony environment, you should use this data instead

To download the whole dataset, a small program in Python is included in this demo. Be warned, this can take a long time (I think Voxforge is throttling the speed to save on costs) and restarts may be neccessary. The python method does check for failed downloads (compares file sizes) and restarts whatever wasn't downloaded completely, so you can run the method 2-3 times to make sure everything is ok.

Alternatively, wou can use a program like wget and enter this command (where "audio" is the dir to save the data to):

    wget -P audio -l 1 -N -nd -c -e robots=off -A tgz -r -np http://www.repository.voxforge1.org/downloads/SpeechCorpus/Trunk/Audio/Main/16kHz_16bit
 
First lets import all the voxforge methods from the python directory. These will need the following libraries installed on your system:
  * numpy - for working with data
  * random, urllib, lxml, os, tarfile, gzip, re, pickle, shutil - these are standard system libraries and anyone should have them
  * scikits.audiolab - to load the audio files from the database (WAV and FLAC files)
  * tqdm - a [simple library for progressbars](https://github.com/noamraph/tqdm) that you can install using pip

In [11]:
import sys

sys.path.append('../python')

from voxforge import *

/usr/local/lib/python2.7/dist-packages/scikits/audiolab/soundio/play.py:48: UserWarning: Could not import alsa backend; most probably, you did not have alsa headers when building audiolab
  warnings.warn("Could not import alsa backend; most probably, "


Ignore any warnings above (I coudn't be bothered to compile audiolab with Alsa). Below you will find the method to download the Voxforge database. You only need to do this once, so you can run it either here or from a console or use wget. Be warned that it takes a long time (as mentioned earlier) so it's a good idea to leave it running over night.

In [ ]:
downloadVoxforgeData('../audio')

## Loading the corpus

Once the data is downloaded and stored in the 'audio' subdir of the main project dir, we can start loading the data into a Python datastructure. There are several methods that can be used for that. The following method will load a file and display its contents:

In [12]:
f=loadFile('../audio/Joel-20080716-qoz.tgz')
print f.props
print f.prompts
print f.data

%xdel f

{'Pronunciation dialect': 'American English', 'File type': 'wav', 'Age Range': 'Adult', 'Speaker Characteristics': '', 'Language': 'EN', 'File Info': '', 'Gender': 'Male', 'Audio Recording Software': 'VoxForge Speech Submission Application', 'Audio card type': 'unknown', 'User Name': 'Joel', 'Sample rate format': '16', 'Number of channels': '1', 'O/S': '', 'Microphone make': 'n/a', 'Path': 'Joel-20080716-qoz', 'Sampling Rate': '48000', 'Microphone type': 'USB Headset mic', 'Recording Information': '', 'Audio card make': 'unknown'}
{'b0076': ['BEFORE', 'PHILIP', 'COULD', 'RECOVER', 'HIMSELF', "JEANNE'S", 'STARTLED', 'GUARDS', 'WERE', 'UPON', 'HIM'], 'b0077': ['IT', 'IS', 'THE', 'NEAREST', 'REFUGE'], 'b0074': ['YET', 'BEHIND', 'THEM', 'THERE', 'WAS', 'ANOTHER', 'AND', 'MORE', 'POWERFUL', 'MOTIVE'], 'b0075': ['IN', 'THAT', 'CASE', 'HE', 'COULD', 'NOT', 'MISS', 'THEM', 'IF', 'HE', 'USED', 'CAUTION'], 'b0081': ['YOU', 'WERE', 'GOING', 'TO', 'LEAVE', 'AFTER', 'YOU', 'SAW', 'ME', 'ON', 'THE',

The loadBySpeaker method will load the whole folder and organize its contents by speakers (as a dictionary). Each utterance contains only the data and the prompts. For this demo, only 30 files are read - as this isn't a method we are going to ultimately use.

In [13]:
corp=loadBySpeaker('../audio', limit=30)

The corpus can also be extended by the phonetic transcription of the utterances using a lexicon file. Voxforge does provide such a file on its website and it is downloaded automatically (if it doesn't already exist).

Note that a single word can have several transcriptions. In the lexicon, these alternatives will have sequential number suffixes added to the word (word, word2, word3, etc), but this particular function will do nothing about that. Choosing the right pronounciation variant has to be done either manually, or by using a more sophisticated program (a pre-trained ASR system) to choose the right version automatically.

In [14]:
addPhonemesSpk(corp,'../data/lex.tgz')

In [15]:
print corp.keys()

spk=corp.keys()[0]

print corp[spk]

%xdel corp

['Bitbrit', 'Perygryne', 'dw96', 'bugsysservant', 'Krellis', 'nalbion', 'camdixon', 'anonymous_9', 'anonymous_8', 'anonymous_5', 'Splitlocked', 'anonymous_7', 'anonymous_6', 'anonymous_1', 'corno1979', 'anonymous_3', 'anonymous_2', 'ductapeguy', 'Dimon', 'poorsquinky', 'anonymous_4', 'anonymous_11', 'anonymous_10', 'rilomino', 'Robertcz', 'joel', 'snblitz', 'inequation', 'pcsnpny', 'Bahoke']
{'b0179': [array([ 0,  0,  0, ..., 29, 18,  3], dtype=int16), ['LET', 'THEM', 'GO', 'OUT', 'AND', 'EAT', 'WITH', 'MY', 'BOYS'], ['l', 'eh', 't', 'dh', 'eh', 'm', 'g', 'ow', 'aw', 't', 'ah', 'n', 'd', 'iy', 't', 'w', 'ih', 'dh', 'm', 'ay', 'b', 'oy', 'z']], 'b0178': [array([  0,   0,   0, ..., -27, -65, -57], dtype=int16), ['ALSO', 'I', 'WANT', 'INFORMATION'], ['ao', 'l', 's', 'ow', 'ay', 'w', 'aa', 'n', 't', 'ih', 'n', 'f', 'ao', 'r', 'm', 'ey', 'sh', 'ah', 'n']], 'b0184': [array([  0,   0,   0, ..., -13, -18,  -7], dtype=int16), ['SUCH', 'THINGS', 'IN', 'HER', 'BRAIN', 'WERE', 'LIKE', 'SO', 'MANY'

## Aligned corpus

As mentioned earlier, this sort or cropus has it's downsides. For one, we don't know when each phoneme occurs so we cannot train the system discriminatavely. While it's still possible, it would be nice if we could start with a simpler example. Another problem is choosing the right pronounciation variant mentioned above.

To solve these issues, an automatic alignement was created using a different ASR system called [Kaldi](http://kaldi-asr.org). This system is a very good ASR solution that implements various types of models. It also contains simple out-of-the-box scripts for training on Voxforge data.

To create the alignments using Kaldi, a working system had to be trained first and what's interesting, the same Voxforge data was used to train the system. How was this done? Well, Kaldi uses (among other things) a classic Gaussian Mixture Model and trains it using the EM algorithm. Initially the alignment is assumed to be even, throughout the file, but as the system is trained iteratively, the model gets better and thus the alignment gets more accurate. The system is trained with gradually better models to achieve even more accurate results and the provided solution here is generated using the "tri3b" model, as described in the scripts.

The alignments in Kaldi are stored in special binary files, but there are simple tools to help convert them into something more easier to use. The type of file chosen for this example is the CTM file, which contains a series of lines in a text file, each line describing a single word or phoneme. The description has 5 columns: encoded file name, unused id (always 1), segment start, segment length and segment text (i.e. word of phoneme name/value). This file was generated using Kaldi, compressed using gzip and stored in 'ali.ctm.gz' in the 'data' directory of this project.

Please note, that the number of files in this aligned set is smaller than the acutal count in the whole Voxforge dataset. This is because there is a small percentage of errors in the database (around a 100 files or so) and some recordings are of such poor quality that Kaldi couldn't generate a reasonable alignemnet for these files. We can simply ignore them here. This, however, doesn't mean that all the alignments present in the CTM are 100% accurate. There can still be mistakes there, but hopefully they are unlikely enough to not cause any issue.

While this file contains everything that we need, it'd be useful to convert it into a datastructure that can be easily used in Python. The convertCTMToAli method is used for that:

In [ ]:
convertCTMToAli('../data/ali.ctm.gz','../data/phones.list','../audio','../data/ali.pklz')

We store the generated datastructure into a gzipped and pickled file, so we don't need to perform this more than once. This file is already included in the repository, so you can skip the step above.

We can read the file like this:

In [16]:
import gzip
import pickle
with gzip.open('../data/ali.pklz') as f:
    ali=pickle.load(f)

In [17]:
print len(ali)

print ali[100].spk
print ali[100].phones
print ali[100].ph_lens
print ali[100].archive
print ali[100].audiofile
print ali[100].data

59274
Aaron
[0, 10, 11, 28, 38, 23, 1, 31, 3, 23, 6, 25, 31, 3, 3, 35, 31, 28, 34, 32, 17, 23, 17, 31, 0]
[480, 70, 70, 50, 70, 70, 90, 90, 30, 60, 170, 90, 90, 90, 80, 50, 120, 60, 80, 100, 40, 60, 80, 140, 720]
Aaron-20080318-ngh
b0346
None


To make things more managable for this demo, we will take 5% of the whole dataset and work using that instead of the whole 80 hours. 5% should give us an amount similar to TIMIT. If you wish to re-run the experiments using the whole dataset, be wary that it takes almost 10GB in memory - you should have enough RAM and VRAM if you intend to use the GPU.

In [ ]:
import random

random.shuffle(ali)

num=int(len(ali)*0.05)

ali=ali[:num]

with gzip.open('../data/ali_p5.pklz','wb') as f:
    pickle.dump(ali,f,pickle.HIGHEST_PROTOCOL)

If you want to compare results to those in this repository, make sure you use the same ali_p5.pklz file as the one already present in the repo already. If you re-generate this file using the commands above, your results will not be comparable.

Please note that the audio data is not yet loaded at this step (it's set to None). To do this, we use the loadAlignedCorpus method. It loads the alignment and the appropriate audio datafile for each utterance:

In [18]:
corp=loadAlignedCorpus('../data/ali_p5.pklz','../audio')

Now we can check if we have all the neccessary data: phonemes, phoneme alignments and data.

In [19]:
print len(corp)

print corp[0].phones
print corp[0].ph_lens
print corp[0].data

samp_num=0
for utt in corp:
    samp_num+=utt.data.size

print str(((samp_num/16000.0)/60.0)/60.0)+' hours'

2963
[0, 10, 18, 25, 21, 9, 20, 3, 23, 29, 11, 27, 30, 3, 23, 35, 12, 8, 34, 21, 18, 22, 13, 23, 31, 13, 23, 9, 17, 31, 29, 11, 21, 14, 0, 17, 38, 13, 23, 37, 34, 30, 13, 27, 0]
[380, 70, 90, 120, 60, 40, 80, 40, 70, 70, 70, 50, 90, 50, 100, 50, 80, 130, 50, 60, 80, 70, 100, 40, 90, 80, 30, 40, 50, 40, 110, 70, 110, 130, 250, 90, 30, 100, 60, 60, 80, 140, 170, 90, 880]
[0 0 0 ..., 0 0 0]
4.01141171875 hours


Here we will save the database to the disk to save time loading it in the future. This isn't strictly neccesary here, unless you want to play with different feature sets and don't feel like repeating the scripts above.

In [ ]:
import cPickle as pickle

with open('../data/ali_w_data.pkl','wb') as f:
    pickle.dump(corp,f,pickle.HIGHEST_PROTOCOL)

This is how you load it:

In [ ]:
with open('../data/ali_w_data.pkl') as f:
    corp=pickle.load(f)

## Feature extraction

To perform a simple test, we will use a standard set of audio features used in many, if not most papers on speech recognition. This set of features will first split each file into a bunch of small chunks of equal size, giving about a 100 of such frames per second. Each chunk will then be converted into a vector of 39 real values. Furhtermore, each vector will be assigned a phonetic class (value from 0..39) thanks to the alignemnt created above. The problem can then be sovled as a simple classification problem that maps a real vector to a phonetic class.

This particular set of features is calculated to match a specification developed in a classic toolkit known as [HTK](http://htk.eng.cam.ac.uk/). All the details on this feature set can be found under the linked repository [here](https://github.com/danijel3/PyHTK). If you want to experiment with this feature set (highly encouraged) please read the description there.

In the code below, we extract the set of features for each utterance and store the results, together with the classification decision for each frame. All the values are stored in large dense numpy arrays to speed up processing. In order to save the utterance information, a third array is created containing the lengths of utterances. This may come in handy for certain tasks later.

In [ ]:
sys.path.append('../PyHTK/python')

import numpy as np
from HTKFeat import MFCC_HTK

from tqdm import *

mfcc=MFCC_HTK()

inputs=[]
utt_sizes=[]
outputs=[]
for utt in tqdm(corp):
    
    feat=mfcc.get_feats(utt.data)
    delta=mfcc.get_delta(feat)
    acc=mfcc.get_delta(delta)
    
    feat=np.hstack((feat,delta,acc))
    utt_len=feat.shape[0]
    
    inputs.append(feat)
    utt_sizes.append(utt_len)
    
    o=[]
    for i in range(len(utt.phones)):
        num=utt.ph_lens[i]/10
        o.extend([utt.phones[i]]*num)
    
    if utt_len-len(o)==1:
        o.append(o[-1])
    
    assert len(o)==utt_len
    
    outputs.extend(o)

inputs=np.vstack(inputs)
utt_sizes=np.array(utt_sizes)
outputs=np.array(outputs)

np.save('../data/mfcc_in.npy',inputs)
np.save('../data/mfcc_out.npy',outputs)
np.save('../data/mfcc_utt.npy',utt_sizes)

## Normalization

While usable as-is, many machine learning models will perform badly if the data isn't standardized. Standarization or normalization stands for making sure that all the samples are distributed to a reasonable scale - usually centered around 0 (with a mean of 0) and spread to a standard deviation of 1. The reason for this is because the data can come from various sources - some people are louder, some are quiter, some have higher pitched voices, some lower, some used a more sensitive microphones than other, etc. That is why the audio between sessions can have various ranges of values. Normalization makes sure that all the recordings are tuned to a similar scale before processing.

To perform normalization we simply need to compute the mean and standard deviation of the given signal and then subtract the mean and divide by the standard deviation (thus making the new mean 0 and new stdev 1). A common question is what signal do we use to perform these calculations? Do we calculate it once for the whole corpus, or once per each utterance? Maybe once per speaker or once per session? Or maybe several times per utterance?

Generally, the longer the signal we do this on, the better (the statistics get more accurate), but performing it only once on the whole corpus doesn't make much sense because of what is written above. The reason we normalize the data is to remove the differences between recording sessions, so at minimum we should normalize each session seperately. In practice, it's easier to just normalize each utterance as they are long enough on their own. This is known as "batch normalization" (where each utterance is one batch).

But this makes one assumption, that the recording conditions don't change significantly throughout the whole utterance. In certain cases, it may actually be a good idea to split the utterance into several parts and normalize them sepeartely, in case the volume changes throught the recording, or maybe there is more than one speaker in a single file. This is solved best by using a technique known as "online normalization" which uses a sliding window to compute the statistics and can react to rapid changes in the values. This is, however, beyond the scope of this simple demo (and shouldn't really be neccessary for this corpus anyway).

This is how bach normalization is performed on our corpus:

In [ ]:
b=0
for utt in tqdm(utt_sizes):
    e=b+utt
    
    f=inputs[b:e]#this is a view, so changes to this are in-place
    f-=np.mean(f)
    f/=np.std(f)
    
    b=e
    
np.save('../data/mfcc_in_norm.npy',inputs)

## Simple classification example

To finish here, we will use a simple SGD classifier from the scikit.learn library to classify the phonemes from the database. First, we will need to split the database into 3 parts: train (80%), dev(10%) and test(10%). The data was already shuffled when we prepared the ali_p5.pklz file so an even split on the data should give good results. If you use the same ali_p5.pklz file as available in the repo, you should be able to repeat the same experiemnts as the one below.

In [1]:
import numpy as np

inputs=np.load('../data/mfcc_in_norm.npy')
outputs=np.load('../data/mfcc_out.npy')

num=inputs.shape[0]

p1=int(num*0.8)
p2=int(num*0.9)

train_idx=range(0,p1)
dev_idx=range(p1,p2)
test_idx=range(p2,num)

Here we create the SGD classifier model. Please note that the settings below work on the version 0.17 of scikit-learn, so it's recommended to upgrade. If you can't, then feel free to modify the settings to something that works for you:

In [2]:
import sklearn
print sklearn.__version__

from sklearn.linear_model import SGDClassifier

model=SGDClassifier(loss='log',n_jobs=-1,verbose=1,n_iter=100)

0.17


Here we train the model. It took 4 minutes for me:

In [3]:
model.fit(inputs[train_idx],outputs[train_idx])

-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 15.43, NNZs: 39, Bias: 12.511262, T: 1150888, Avg. loss: 0.382643
Total training time: 0.70 seconds.
-- Epoch 2
Norm: 6.68, NNZs: 39, Bias: -22.124768, T: 1150888, Avg. loss: 0.168558
Total training time: 0.75 seconds.
-- Epoch 2
Norm: 10.01, NNZs: 39, Bias: -65.304629, T: 1150888, Avg. loss: 0.095711
Total training time: 0.75 seconds.
-- Epoch 2
Norm: 9.26, NNZs: 39, Bias: -59.676164, T: 1150888, Avg. loss: 0.122729
Total training time: 0.76 seconds.
-- Epoch 2
Norm: 6.47, NNZs: 39, Bias: -5.392906, T: 1150888, Avg. loss: 0.224968
Total training time: 0.78 seconds.
-- Epoch 2
Norm: 6.15, NNZs: 39, Bias: -26.944058, T: 1150888, Avg. loss: 0.099888
Total training time: 0.79 seconds.
-- Epoch 2
Norm: 8.24, NNZs: 39, Bias: -48.540849, T: 1150888, Avg. loss: 0.148078
Total training time: 0.79 seconds.
-- Epoch 2
Norm: 4.51, NNZs: 39, Bias: -4.148019, T: 

[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  4.6min finished


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=100, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=1, warm_start=False)

Here we get about ~56% accuracy which is pretty bad for phoneme recogntion. In other notebooks, we will try to improve on that.

In [4]:
acc=model.score(inputs[test_idx],outputs[test_idx])
print 'Accuracy: {:%}'.format(acc)

Accuracy: 56.418348%
